In [1]:
from bw2data import projects, databases, Database
from bw2io import Migration
import re

In [2]:
projects.set_current("EXIOBASE regionalized case study")

In [3]:
if "Oil seeds" in databases:
    del databases['Oil seeds']

# Extracting data

In [4]:
from bw2io.extractors import ExcelExtractor
from copy import deepcopy
from bw2io.importers.base_lci import LCIImporter

In [5]:
import pandas as pd

In [6]:
df = pd.read_excel("data/Foreground direct.xlsx", index_col=[0,1,2,3], header=[0,1,2,3,4,5,6])
df

Product                                                                         Barley cultivation [barley]/UA  \
Unit                                                                                                        kg   
                                                                                                  2252.000000    
                                                                                            Unnamed: 4_level_3   
Category                                                                    \_5 Veg Oil case\_crop cultivation   
Comment                                                                                     Unnamed: 4_level_5   
Type                                                                                                  Material   
Compartment Product/Substance                          Sub-compartment Unit                                      
NaN         Barley cultivation [barley]/UA             NaN             kg                                  NaN   
            Soybean cultivation [soybean]/BR           NaN             kg                                  NaN   
            Oil palm cultivation [FFB]/MY              NaN             kg                                  NaN   
            Rapeseed cultivation [rapeseed]/EU27       NaN             kg                                  NaN   
            Sunflower cultivation [sunflower seeds]/UA NaN             kg                                  NaN   
...                                                                                                        ...   
Air         Methane                                    NaN             kg                                  NaN   
            Nitrogen oxides                            NaN             kg                             1.928571   
Water       Nitrate                                    NaN             kg                           112.737833   
            Phosphorus                                 NaN             kg                                  NaN   
Raw         Occupation, arable                         NaN             ha a                           1.000000   

Product                                                                       Soybean cultivation [soybean]/BR  \
Unit                                                                                                        kg   
                                                                                                  2863.000000    
                                                                                            Unnamed: 5_level_3   
Category                                                                    \_5 Veg Oil case\_crop cultivation   
Comment                                                                                     Unnamed: 5_level_5   
Type                                                                                                  Material   
Compartment Product/Substance                          Sub-compartment Unit                                      
NaN         Barley cultivation [barley]/UA             NaN             kg                                  NaN   
            Soybean cultivation [soybean]/BR           NaN             kg                                  NaN   
            Oil palm cultivation [FFB]/MY              NaN             kg                                  NaN   
            Rapeseed cultivation [rapeseed]/EU27       NaN             kg                                  NaN   
            Sunflower cultivation [sunflower seeds]/UA NaN             kg                                  NaN   
...                                                                                                        ...   
Air         Methane                                    NaN             kg                                  NaN   
            Nitrogen oxides                            NaN             kg                               0.0000   
Water    

In [7]:
# rename column levels, drop empty level
df.index.names = ["Product Compartment", "Product Name", "Product Sub-compartment", "Product Unit"]
df.columns.names = ["Process Name", "Process Unit", "Process Amount", "Delete", "Process Category", "Process Comment", "Process Type"]
df.columns = df.columns.droplevel("Delete")
df

Process Name                                                                                            Barley cultivation [barley]/UA  \
Process Unit                                                                                                                        kg   
Process Amount                                                                                                            2252.000000    
Process Category                                                                                    \_5 Veg Oil case\_crop cultivation   
Process Comment                                                                                                     Unnamed: 4_level_5   
Process Type                                                                                                                  Material   
Product Compartment Product Name                               Product Sub-compartment Product Unit                                      
NaN                 Barley cultivation [barley]/UA             NaN                     kg                                          NaN   
                    Soybean cultivation [soybean]/BR           NaN                     kg                                          NaN   
                    Oil palm cultivation [FFB]/MY              NaN                     kg                                          NaN   
                    Rapeseed cultivation [rapeseed]/EU27       NaN                     kg                                          NaN   
                    Sunflower cultivation [sunflower seeds]/UA NaN                     kg                                          NaN   
...                                                                                                                                ...   
Air                 Methane                                    NaN                     kg                                          NaN   
                    Nitrogen oxides                            NaN                     kg                                     1.928571   
Water               Nitrate                                    NaN                     kg                                   112.737833   
                    Phosphorus                                 NaN                     kg                                          NaN   
Raw                 Occupation, arable                         NaN                     ha a                                   1.000000   

Process Name                                                                                          Soybean cultivation [soybean]/BR  \
Process Unit                                                                                                                        kg   
Process Amount                                                                                                            2863.000000    
Process Category                                                                                    \_5 Veg Oil case\_crop cultivation   
Process Comment                                                                                                     Unnamed: 5_level_5   
Process Type                                                                                                                  Material   
Product Compartment Product Name                               Product Sub-compartment Product Unit                                      
NaN                 Barley cultivation [barley]/UA             NaN                     kg                                          NaN   
                    Soybean cultivation [soybean]/BR           NaN                     kg                                          NaN   
                    Oil palm cultivation [FFB]/MY              NaN                     kg                                          NaN   
                    Rapeseed cultivation [rapeseed]/EU27       NaN                     kg                                          NaN 

In [8]:
# un-pivot to remove NaN values
df_long = df.melt(ignore_index=False, value_name="Exchange Amount").dropna()
df_long

Process Name  \
Product Compartment Product Name                                       Product Sub-compartment Product Unit                                                      
NaN                 Link to: Market for arable land {GLO}              NaN                     ha a                             Barley cultivation [barley]/UA   
                    Link to: Road transport {UA} 16-32 t truck         NaN                     tkm                              Barley cultivation [barley]/UA   
                    Link to: N-fertiliser, urea, as N {UA}             NaN                     kg                               Barley cultivation [barley]/UA   
                    Link to: N-fertiliser, ammonium nitrate, as N {UA} NaN                     kg                               Barley cultivation [barley]/UA   
                    Link to: N-fertiliser, ammonium sulphate, as N ... NaN                     kg                               Barley cultivation [barley]/UA   
...                                                                                                                                                        ...   
                    Barley to generic market for feed [energy feed]... NaN                     MJ               Utilisation of FFA as feed [FFA treatment]/GLO   
                    Soybean oil mill [SBM]/BR                          NaN                     kg            Soybean meal to generic market for feed [prote...   
                    Barley to generic market for feed [energy feed]... NaN                     MJ            Soybean meal to generic market for feed [prote...   
                    Barley cultivation [barley]/UA                     NaN                     kg            Barley to generic market for feed [energy feed...   
                    Soybean meal to generic market for feed [protei... NaN                     kg            Barley to generic market for feed [energy feed...   

                                                                                                            Process Unit  \
Product Compartment Product Name                                       Product Sub-compartment Product Unit                
NaN                 Link to: Market for arable land {GLO}              NaN                     ha a                   kg   
                    Link to: Road transport {UA} 16-32 t truck         NaN                     tkm                    kg   
                    Link to: N-fertiliser, urea, as N {UA}             NaN                     kg                     kg   
                    Link to: N-fertiliser, ammonium nitrate, as N {UA} NaN                     kg                     kg   
                    Link to: N-fertiliser, ammonium sulphate, as N ... NaN                     kg                     kg   
...                                                                                                                  ...   
                    Barley to generic market for feed [energy feed]... NaN                     MJ                     kg   
                    Soybean oil mill [SBM]/BR                          NaN                     kg                     kg   
                    Barley to generic market for feed [energy feed]... NaN                     MJ                     kg   
                    Barley cultivation [barley]/UA                     NaN                     kg                     MJ   
                    Soybean meal to generic market for feed [protei... NaN                     kg                     MJ   

                                                                                                             Process Amount  \
Product Compartment Product Name                                       Product Sub-compartment Product Unit                   
NaN                 Link to: Market for arable land {GLO}              NaN                     ha a              2252.00000   
                    Link to: Road transport {

In [9]:
def df_to_database(df):
    
    def split_name_and_location(string):
        if len(re.findall("/",string)) == 1 and "g/l" not in string:
            return string.split("/")[0], string.split("/")[1]
        elif "{" in string:
            return re.sub(" {.*?}", "", string), re.search("{(.*?)}", string).group(1)
        else:
            return string, "GLO"
    
    db = []
    processes = df["Process Name"].unique()
    for name in processes:
        
        # parse process metadata
        process = {"code": name}
        process["name"], process["location"] = split_name_and_location(name)               
        
        # parse exchanges
        df_ex = df.query(f"`Process Name`=='{name}'") 
        process["exchanges"] = []
        for index, row in df_ex.iterrows():
            
            # more process metadata
            if len(process["exchanges"])==0:
                process["unit"] = row["Process Unit"]
                process["category"] = row["Process Category"]
                if row["Process Comment"] != "Unnamed: 4_level_5":
                    process["comment"] = row["Process Comment"]
                process["kind"] = row["Process Type"]
                process["exchanges"].append({
                    "type":"production", 
                    "amount": row["Process Amount"], 
                    "code":process["code"],
                    "name":process["name"],
                    "location":process["location"],
                    "unit": process["unit"],
                })
            
            # exchange data
            ex = {
                "code": index[1],
                "amount": row["Exchange Amount"],
                "type": "technosphere",
                "unit": index[-1],
            }
            ex["name"], ex["location"] = split_name_and_location(index[1])
            
            # specific to biosphere exchanges
            if not pd.isna(index[0]):
                ex["categories"] = index[0]
                ex["type"] = "biosphere"
            if not pd.isna(index[2]):
                ex["categories"] = (ex["categories"], + index[2])
                
            process["exchanges"].append(ex)
            
        db.append(process)
        
    return db

db = df_to_database(df_long)
db[0]

{'code': 'Barley cultivation [barley]/UA',
 'name': 'Barley cultivation [barley]',
 'location': 'UA',
 'exchanges': [{'type': 'production',
   'amount': 2252.0,
   'code': 'Barley cultivation [barley]/UA',
   'name': 'Barley cultivation [barley]',
   'location': 'UA',
   'unit': 'kg'},
  {'code': 'Link to: Market for arable land {GLO}',
   'amount': 0.89,
   'type': 'technosphere',
   'unit': 'ha a',
   'name': 'Link to: Market for arable land',
   'location': 'GLO'},
  {'code': 'Link to: Road transport {UA} 16-32 t truck',
   'amount': 119.17193191325892,
   'type': 'technosphere',
   'unit': 'tkm',
   'name': 'Link to: Road transport 16-32 t truck',
   'location': 'UA'},
  {'code': 'Link to: N-fertiliser, urea, as N {UA}',
   'amount': 8.763862591290234,
   'type': 'technosphere',
   'unit': 'kg',
   'name': 'Link to: N-fertiliser, urea, as N',
   'location': 'UA'},
  {'code': 'Link to: N-fertiliser, ammonium nitrate, as N {UA}',
   'amount': 49.49959426562078,
   'type': 'technosphe

In [10]:
df_inter = pd.read_excel("data/Foreground intermediate.xlsx", index_col=[0,1,2,3], header=[0,1,2,3,4,5,6])
df_inter

Product                                                                             Link to: N-fertiliser, ammonium nitrate, as N {AR}  \
Unit                                                                                                                               ton   
                                                                                                                                     1   
                                                                                                                    Unnamed: 4_level_3   
Category                                                                                \_5 Veg Oil case\_Link to background: Exiobase   
Comment                                                                                                     Link to Exiobase v3.3.13b2   
Type                                                                                                                          Material   
Compartment Product/Substance                                  Sub-compartment Unit                                                      
NaN         _20 Mining of coal and lignite; extraction of p... NaN             ton                                                 NaN   
            _20 Mining of coal and lignite; extraction of p... NaN             ton                                                 NaN   
            _20 Mining of coal and lignite; extraction of p... NaN             ton                                                 NaN   
            _20 Mining of coal and lignite; extraction of p... NaN             ton                                                 NaN   
            _22 Extraction of natural gas and services rela... NaN             ton                                                 NaN   
...                                                                                                                                ...   
            Electricity mix {WA} (product market, hybrid un... NaN             kWh                                                 NaN   
            Electricity mix {RU} (product market, hybrid un... NaN             kWh                                                 NaN   
            Electricity mix {WE} (product market, hybrid un... NaN             kWh                                                 NaN   
            Electricity mix {US} (product market, hybrid un... NaN             kWh                                                 NaN   
            Arable land, as ha*year-eq. (linked)               NaN             ha a                                                NaN   

Product                                                                             Link to: N-fertiliser, ammonium nitrate, as N {BR}  \
Unit                                                                                                                               ton   
                                                                                                                                     1   
                                                                                                                    Unnamed: 5_level_3   
Category                                                                                \_5 Veg Oil case\_Link to background: Exiobase   
Comment                                                                                                     Link to Exiobase v3.3.13b2   
Type                                                                                                                          Material   
Compartment Product/Substance                                  Sub-compartment Unit                                                      
NaN         _20 Mining of coal and lignite; extraction of p... NaN             ton                                                 NaN   
            _20 Mining of coal and lignite; extraction of p... NaN             ton                                                 NaN 

In [11]:
# rename column levels, drop empty level
df_inter.index.names = ["Product Compartment", "Product Name", "Product Sub-compartment", "Product Unit"]
df_inter.columns.names = ["Process Name", "Process Unit", "Process Amount", "Delete", "Process Category", "Process Comment", "Process Type"]
df_inter.columns = df_inter.columns.droplevel("Delete")
df_inter

# un-pivot to remove NaN values
df_long = df_inter.melt(ignore_index=False, value_name="Exchange Amount").dropna()
df_long

Process Name  \
Product Compartment Product Name                                       Product Sub-compartment Product Unit                                                      
NaN                 _61 N-fertiliser {WL} (product market, hybrid u... NaN                     ton           Link to: N-fertiliser, ammonium nitrate, as N ...   
                    _61 N-fertiliser {BR} (product market, hybrid u... NaN                     ton           Link to: N-fertiliser, ammonium nitrate, as N ...   
                    _61 N-fertiliser {WL} (product market, hybrid u... NaN                     ton           Link to: N-fertiliser, ammonium nitrate, as N ...   
                    _61 N-fertiliser {DE} (product market, hybrid u... NaN                     ton           Link to: N-fertiliser, ammonium nitrate, as N ...   
                    _61 N-fertiliser {ID} (product market, hybrid u... NaN                     ton           Link to: N-fertiliser, ammonium nitrate, as N ...   
...                                                                                                                                                        ...   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                      Link to: Steel scrap to recycling {BR}   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                      Link to: Steel scrap to recycling {CO}   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                      Link to: Steel scrap to recycling {ID}   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                      Link to: Steel scrap to recycling {MY}   
                    Arable land, as ha*year-eq. (linked)               NaN                     ha a                      Link to: Market for arable land {GLO}   

                                                                                                            Process Unit  \
Product Compartment Product Name                                       Product Sub-compartment Product Unit                
NaN                 _61 N-fertiliser {WL} (product market, hybrid u... NaN                     ton                   ton   
                    _61 N-fertiliser {BR} (product market, hybrid u... NaN                     ton                   ton   
                    _61 N-fertiliser {WL} (product market, hybrid u... NaN                     ton                   ton   
                    _61 N-fertiliser {DE} (product market, hybrid u... NaN                     ton                   ton   
                    _61 N-fertiliser {ID} (product market, hybrid u... NaN                     ton                   ton   
...                                                                                                                  ...   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                   ton   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                   ton   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                   ton   
                    _73 Re-processing of secondary steel into new s... NaN                     ton                   ton   
                    Arable land, as ha*year-eq. (linked)               NaN                     ha a                 ha a   

                                                                                                             Process Amount  \
Product Compartment Product Name                                       Product Sub-compartment Product Unit                   
NaN                 _61 N-fertiliser {WL} (product market, hybrid u... NaN                     ton                        1   
                    _61 N-fertiliser {BR} (pr

In [12]:
db_inter = df_to_database(df_long)
db_inter[0]

{'code': 'Link to: N-fertiliser, ammonium nitrate, as N {AR}',
 'name': 'Link to: N-fertiliser, ammonium nitrate, as N',
 'location': 'AR',
 'exchanges': [{'type': 'production',
   'amount': 1,
   'code': 'Link to: N-fertiliser, ammonium nitrate, as N {AR}',
   'name': 'Link to: N-fertiliser, ammonium nitrate, as N',
   'location': 'AR',
   'unit': 'ton'},
  {'code': '_61 N-fertiliser {WL} (product market, hybrid units)',
   'amount': 1.0,
   'type': 'technosphere',
   'unit': 'ton',
   'name': '_61 N-fertiliser (product market, hybrid units)',
   'location': 'WL'}],
 'unit': 'ton',
 'category': '\\_5 Veg Oil case\\_Link to background: Exiobase',
 'comment': 'Link to Exiobase v3.3.13b2',
 'kind': 'Material'}

# Creating an `importer` object

In [13]:
ei = LCIImporter("Oil seeds")
ei.data = db + db_inter
del ei.strategies[0]
del ei.strategies[0]

In [14]:
from bw2io.strategies import normalize_units

In [15]:
ei.apply_strategy(normalize_units)

Applying strategy: normalize_units


We have one more unit to fix - `ha a`, or the occupation of one hectare for one year. Normally we could do `ei.migrate("default-units")`, which would convert this unit to what our base flow list expects (occupation of square meter - year), but this would change all our tonnes to kilograms, meaning we would have the wrong unit to link to exiobase. So we treat this as a special case migration.

('ha a', 'square meter-year', 1e4)

In [16]:
Migration("hecatare-units").write({
    'fields': ['unit'],
    'data': [
        (
            ('ha a',),
            {'unit': 'square meter-year', 'multiplier': 1e4}
        )
    ]
}, 'Change only `ha a` units')

ei.migrate("hecatare-units")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [17]:
{ds['unit'] for ds in ei.data}

{'cubic meter',
 'kilogram',
 'kilowatt hour',
 'megajoule',
 'square meter-year',
 'ton',
 'ton kilometer'}

EXIOBASE uses `Meuro` instead of `MEUR2011`, let's change this.

In [18]:
databases

Databases dictionary with 3 object(s):
	EXIOBASE 3.3.18 hybrid
	EXIOBASE 3.3.18 hybrid biosphere
	biosphere3

In [19]:
{ds['unit'] for ds in Database("EXIOBASE 3.3.18 hybrid")}

{'Meuro', 'TJ', 'tonnes', 'tonnes (service)'}

In [20]:
Migration("MEUR2011").write({
    'fields': ['unit'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('MEUR2011',),
            # Second element is new values to substitute
            {
                'unit': 'Meuro',
            }
        )
    ]
}, 'Change Euro unit label')

ei.migrate("MEUR2011")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [21]:
Migration("units").write({
    'fields': ['unit'],
    'data': [
        (
            ('megajoule',),
            {'unit': 'TJ', 'multiplier': 1e-6}
        ),
        (
            ('kilowatt hour',),
            {'unit': 'TJ', 'multiplier': 3.6/1e6}
        ),
                (
            ('kilogram',),
            {'unit': 'tonnes', 'multiplier': 1e-3}
        ),
        (
            ('ton',),
            {'unit': 'tonnes'}
        ),
    ]
}, 'Change MJ, kWh, kg, ton')

ei.migrate("units")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [22]:
{ds['unit'] for ds in ei.data}

{'TJ', 'cubic meter', 'square meter-year', 'ton kilometer', 'tonnes'}

## `database`

We can label each dataset with out chosen database name

In [23]:
from bw2io.strategies import add_database_name
from functools import partial

In [24]:
ei.apply_strategy(partial(add_database_name, name=ei.db_name))

Applying strategy: add_database_name


In [25]:
db[0]

{'code': 'Barley cultivation [barley]/UA',
 'name': 'Barley cultivation [barley]',
 'location': 'UA',
 'exchanges': [{'type': 'production',
   'amount': 2.2520000000000002,
   'code': 'Barley cultivation [barley]/UA',
   'name': 'Barley cultivation [barley]',
   'location': 'UA',
   'unit': 'tonnes',
   'loc': 2.2520000000000002},
  {'code': 'Link to: Market for arable land {GLO}',
   'amount': 8900.0,
   'type': 'technosphere',
   'unit': 'square meter-year',
   'name': 'Link to: Market for arable land',
   'location': 'GLO',
   'loc': 8900.0},
  {'code': 'Link to: Road transport {UA} 16-32 t truck',
   'amount': 119.17193191325892,
   'type': 'technosphere',
   'unit': 'ton kilometer',
   'name': 'Link to: Road transport 16-32 t truck',
   'location': 'UA'},
  {'code': 'Link to: N-fertiliser, urea, as N {UA}',
   'amount': 0.008763862591290235,
   'type': 'technosphere',
   'unit': 'tonnes',
   'name': 'Link to: N-fertiliser, urea, as N',
   'location': 'UA',
   'loc': 0.008763862591

# Internal linking 

We can now apply the "generic" default strategies.

In [26]:
ei.apply_strategies()

Applying strategy: assign_only_product_as_production
Applying strategy: strip_biosphere_exc_locations
Applied 2 strategies in 0.00 seconds


We are ready to start thinking about internal linking. Let's see if the codes will match up, first by checking whether they are unique, and then by looking at them manually

In [27]:
dataset_codes = {ds['name'] for ds in ei.data}
exchange_codes = {exc.get('name') for ds in ei.data for exc in ds['exchanges']}
len(dataset_codes), len(exchange_codes), len(dataset_codes.intersection(exchange_codes))

(90, 120, 90)

In [28]:
ei.match_database(fields=['code'])

Applying strategy: link_iterable_by_fields


In [29]:
ei.statistics()

258 datasets
921 exchanges
488 unlinked exchanges
  Type biosphere: 10 unique unlinked exchanges
  Type technosphere: 108 unique unlinked exchanges


(258, 921, 488)

## `biosphere` exchanges

How many can we link without changing anything?

In [30]:
ei.match_database("biosphere3", fields=['name', 'categories'])

Applying strategy: link_iterable_by_fields


In [31]:
ei.statistics()

258 datasets
921 exchanges
456 unlinked exchanges
  Type biosphere: 3 unique unlinked exchanges
  Type technosphere: 108 unique unlinked exchanges


(258, 921, 456)

Sweet, only 3 to fix manually. Let's look at them.

In [32]:
[ex for ex in ei.unlinked if ex["type"]=="biosphere"]

[{'code': 'Carbon dioxide',
  'amount': 0.0,
  'type': 'biosphere',
  'unit': 'tonnes',
  'name': 'Carbon dioxide',
  'categories': 'Air',
  'loc': 0.0},
 {'code': 'Occupation, arable',
  'amount': 10000.0,
  'type': 'biosphere',
  'unit': 'square meter-year',
  'name': 'Occupation, arable',
  'categories': 'Raw',
  'loc': 10000.0},
 {'code': 'Methane',
  'amount': 8.683129949999998e-06,
  'type': 'biosphere',
  'unit': 'tonnes',
  'name': 'Methane',
  'categories': 'Air',
  'loc': 8.683129949999998e-06}]

`Carbon dioxide` and `Methane`, both of which need to be labeled fossil (or not) to get a match. `Occupation, arable` is a shortened version of what we have in our master flow list:

In [33]:
[x for x in Database("biosphere3") if x['name'].lower().startswith('occupation, arable')]

['Occupation, arable, conventional tillage (obsolete)' (square meter, None, ('natural resource', 'land')),
 'Occupation, arable, reduced tillage (obsolete)' (square meter, None, ('natural resource', 'land')),
 'Occupation, arable land, unspecified use' (square meter-year, None, ('natural resource', 'land')),
 'Occupation, arable, conservation tillage (obsolete)' (square meter, None, ('natural resource', 'land'))]

In [34]:
Migration("oil-bio").write({
    'fields': ['name'],
    'data': [
        (('Carbon dioxide',), {'name': 'Carbon dioxide, fossil',}),
        (('Methane',), {'name': 'Methane, fossil',}),
        (('occupation, arable',), 
         {'name': 'Occupation, arable land, unspecified use', 'categories': ('natural resource', 'land')}),
    ]
}, 'Change some biosphere flows in oil seeds foreground')

ei.migrate("oil-bio")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [35]:
ei.match_database("biosphere3", fields=['name', 'categories'])

Applying strategy: link_iterable_by_fields


In [36]:
ei.statistics()

258 datasets
921 exchanges
443 unlinked exchanges
  Type technosphere: 108 unique unlinked exchanges


(258, 921, 443)

This isn't actually correct - there are still unlinked biosphere exchanges, they are just mislabelled for now :)

## `name`

Let's clean up some names, starting with this pattern: `_64 Manufacture of rubber and plastic products (25)  (product market, hybrid units)`.

In [37]:
test_string = '_64 Manufacture of rubber and plastic products (25)  (product market, hybrid units)'

In [38]:
numeric_start = re.compile("^[0-9_]\d\d")

In [39]:
numeric_start.findall(test_string)

['_64']

In [40]:
test_string = '_64 Manufacture of rubber and plastic products (25)'

In [41]:
numeric_end = re.compile("\(\d\d\)$")

In [42]:
numeric_end.findall(test_string)

['(25)']

In [43]:
def clean_name(name):
    name = name.replace("(product market, hybrid units)", "").replace("Link to: ", "").strip()
    prefix = numeric_start.findall(name)
    if prefix:
        name = name.replace(prefix[0], '').strip()
    #suffix = numeric_end.findall(name)
    #if suffix:
    #    name = name.replace(suffix[0], '').strip()
    return name

def clean_names(data):
    for ds in data:
        ds['name'] = clean_name(ds['name'])
        for exc in ds['exchanges']:
            if exc.get('name'):
                exc['name'] = clean_name(exc['name'])
    return data

In [44]:
ei.apply_strategy(clean_names)

Applying strategy: clean_names


Change "Electricity Mix" and "Electricity Market" to "Production of electricity by gas" because 3.3.18 does not contain mixes.

In [45]:
Migration("what-is-electricity").write({
    'fields': ['name'],
    'data': [
        (('Electricity  Market',), {'name': 'Electricity mix',}),
        (('Electricity  market',), {'name': 'Electricity mix',}),
    ]
}, "Let's try being consistent, just to see how it feels")

ei.migrate("what-is-electricity")

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


In [46]:
ei.match_database(fields=['name', 'location', 'unit'])

Applying strategy: link_iterable_by_fields


In [47]:
ei.statistics()

258 datasets
921 exchanges
443 unlinked exchanges
  Type technosphere: 108 unique unlinked exchanges


(258, 921, 443)

In [48]:
[ds for ds in ei.data if "Electricity Mix" in ds["name"]]

[]

## Linking against EXIOBASE

First try, don't expect great success.

In [49]:
ei.match_database("EXIOBASE 3.3.18 hybrid", fields=['name', 'location'])

Applying strategy: link_iterable_by_fields


In [50]:
ei.statistics()

258 datasets
921 exchanges
11 unlinked exchanges
  Type technosphere: 10 unique unlinked exchanges


(258, 921, 11)

In [51]:
ei.write_excel(only_unlinked=True)

Wrote matching file to:
/home/jupyter-benjamin/.local/share/Brightway3/EXIOBASE-regionalized-case-study.9666a926/output/db-matching-Oil-seeds-unlinked.xlsx


'/home/jupyter-benjamin/.local/share/Brightway3/EXIOBASE-regionalized-case-study.9666a926/output/db-matching-Oil-seeds-unlinked.xlsx'

## Specific issues

The flow `Arable land, as ha*year-eq.` and the activity `Link to: Market for arable land {GLO}` are removed, as we don't know where they occur, and are included to use with an indirect land use model.

In [52]:
def remove_iluc_land(data):
    EXCLUDED = ('Arable land, as ha*year-eq. (linked)', 'Market for arable land')
    data = [ds for ds in data if ds['name'] not in EXCLUDED]
    for ds in data:
        ds['exchanges'] = [exc for exc in ds['exchanges'] if exc.get('name') not in EXCLUDED]
    return data

In [53]:
ei.apply_strategy(remove_iluc_land)

Applying strategy: remove_iluc_land


Just for laughs (!?), there are some exchanges where activities where some exchanges are linked in kilograms, and others in tons. So let's fix exchanges in kilograms and switch them to tons when that will produce a link.

In [54]:
def switch_exchange_units_when_helpful(data):
    lookup = {(ds['name'], ds['unit']) for ds in data}
    
    for ds in data:
        for exc in ds['exchanges']:
            if exc.get('input'):
                continue
            if exc['unit'] == 'kilogram' and (exc['name'], 'ton') in lookup:
                exc['unit'] = 'ton'
                exc['amount'] /= 1000
    return data

In [55]:
ei.apply_strategy(switch_exchange_units_when_helpful)

Applying strategy: switch_exchange_units_when_helpful


In [56]:
ei.match_database(fields=['name', 'location', 'unit'])

Applying strategy: link_iterable_by_fields


In [57]:
ei.statistics()

257 datasets
913 exchanges
10 unlinked exchanges
  Type technosphere: 9 unique unlinked exchanges


(257, 913, 10)

Finally, we will have another problem with the electricity mixes - we have both production exchanges and technosphere exchanges we are the same thing. As their signs are flipped, they will cancel each other out, making our technosphere matrix singular:

In [58]:
import pprint

for ds in ei.data:
    if ds['name'] == 'Electricity mix' and len(ds['exchanges']) > 1:
        pprint.pprint(ds)
        break

We can just drop this duplicative technosphere exchange.

In [59]:
def drop_duplicate_production_exchange(data):
    for ds in data:
        if ds['name'] == 'Electricity mix' and len(ds['exchanges']) > 1:
            ds['exchanges'] = [exc for exc in ds['exchanges'] if exc['type'] == 'production']
    return data

In [60]:
ei.apply_strategy(drop_duplicate_production_exchange)

Applying strategy: drop_duplicate_production_exchange


Finally, there are no electricity markets so use natural gas for all of them as a proxy for now.

In [61]:
Migration("electricity-mix-to-gas").write({
    'fields': ['name'],
    'data': [
        (('Electricity mix',), {'name': 'Production of electricity by gas',}),
    ]
}, "Let's try being consistent, just to see how it feels")

ei.migrate("electricity-mix-to-gas")
ei.match_database("EXIOBASE 3.3.18 hybrid", fields=['name', 'location'])

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields


In [62]:
ei.statistics()

257 datasets
913 exchanges
1 unlinked exchanges
  Type technosphere: 1 unique unlinked exchanges


(257, 913, 1)

In [63]:
list(ei.unlinked)[0]

{'code': 'Treatment of shells',
 'amount': 5.0132999999999996e-05,
 'type': 'technosphere',
 'unit': 'tonnes',
 'name': 'Treatment of shells',
 'location': 'GLO',
 'loc': 5.0132999999999996e-05}

Drop "Treatment of shells" because it is not used anywhere.

In [64]:
ei.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [66]:
ei.write_database()

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 04/26/2022 08:07:40
  Finished: 04/26/2022 08:07:40
  Total time elapsed: 00:00:00
  CPU %: 111.90
  Memory %: 0.16
Created database: Oil seeds


Brightway2 SQLiteBackend: Oil seeds